# Menjalankan Hasil Model Training

Pada notebook ini kalian akan diarahkan untuk menjalankan model yang telah kalian dapatkan dari proses training nih! Adapun library yang dibutuhkan untuk notebook ini adalah

<ul>
    <li>Pandas</li>
    <li>Numpy</li>
    <li>OpenCV</li>
    <li>Mediapipe</li>
    <li>CSV</li>
    <li>Pickle</li>
    <li>Scikit Learn</li>
</ul>

In [17]:
# import library and framework
import pandas as pd
from sklearn.svm import SVC
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import cv2
import mediapipe as mp
import os
import pickle

## 1. Membaca Model ML yang Telah Disimpan dalam Bentuk .sav

In [19]:
# tentukan path dari model hasil training kalian
model_path = './model/mymodel.sav'

with open(model_path, 'rb') as file:
    action_model = pickle.load(file)

c:\Users\HP\anaconda3\envs\media-pipe\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## 2. Jalankan Saja Programnya
Silakan kalian jalankan program berikut dan peragakan semua label yang telah kalian latih! pastikan semua dapat dibaca oleh komputer. Bila bisa semua label, selamat! kalian telah berhasil mengembangkan project CV yang kecee ini! Untuk lebih jelasnya mengenai apasaja yang ada pada Computer Vision, dapat kalian pelajari pada sesi pembelajaran Computer Vision, sampai jumpa lagi!

In [22]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# Define global variable for hand keypoints data
hand_keypoint_data = np.array([])

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    # Read until video is completed
  while cap.isOpened():
    # Capture frame-by-frame
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue
    
    try:
      # To improve performance, optionally mark the image as not writeable to
      # pass by reference.
      image.flags.writeable = False
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = hands.process(image)

      # Draw the hand annotations on the image.
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
          mp_drawing.draw_landmarks(
              image,
              hand_landmarks,
              mp_hands.HAND_CONNECTIONS,
              mp_drawing_styles.get_default_hand_landmarks_style(),
              mp_drawing_styles.get_default_hand_connections_style())
      
      # Checking keypoints if complete will do this block of code
      if len(results.multi_hand_landmarks[0].landmark) >= 21:
        # define variable for centering and scaling process
        centering = np.array([])
        scaling = np.array([])

        # Centering X coordinate Process
        for indexPoint in range(21):
          centering = np.append(centering, (
            results.multi_hand_landmarks[0].landmark[0].x - results.multi_hand_landmarks[0].landmark[indexPoint].x))

        # Centering Y coordinate Process
        for indexPoint in range(21):
          centering = np.append(centering, (
            results.multi_hand_landmarks[0].landmark[0].y - results.multi_hand_landmarks[0].landmark[indexPoint].y))

        centering = centering.reshape(2, 21)
        
        # Scaling Process
        for indexIter in range(2):
          for jointIter in range(21):
            scaling = np.append(scaling, centering[indexIter][jointIter] / np.max(
              np.absolute(centering[indexIter])) * 320)
        
        # Normalization Process
        for jointIter in range(42):
          hand_keypoint_data = np.append(hand_keypoint_data, (scaling[jointIter] + 320))

        # Write spatiodata from hand keypoints coordinate
        if len(hand_keypoint_data) >= 210:
          labels = [file[5:-4] for file in os.listdir('data')]
           
          prediction = action_model.predict([hand_keypoint_data])

          deletedIndex = np.arange(42)
          hand_keypoint_data = np.delete(hand_keypoint_data, deletedIndex)

          cv2.putText(image,f'{labels[prediction[0]]}', (10,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (100, 255, 0), 3, cv2.LINE_AA)
         
    except Exception as e:
      continue

    finally:
      # Show the result
      cv2.imshow('Result', image)

      if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()